In [ ]:
!unzip "/content/drive/MyDrive/glacomadataset2.zip"

Archive:  /content/drive/MyDrive/glacomadataset2.zip
   creating: glacomadataset2/
   creating: glacomadataset2/glaucoma/
  inflating: glacomadataset2/glaucoma/image100prime.jpg  
  inflating: glacomadataset2/glaucoma/image101prime.jpg  
  inflating: glacomadataset2/glaucoma/image102prime.jpg  
  inflating: glacomadataset2/glaucoma/image103prime.jpg  
  inflating: glacomadataset2/glaucoma/image104prime.jpg  
  inflating: glacomadataset2/glaucoma/image105prime.jpg  
  inflating: glacomadataset2/glaucoma/image106prime.jpg  
  inflating: glacomadataset2/glaucoma/image107prime.jpg  
  inflating: glacomadataset2/glaucoma/image108prime.jpg  
  inflating: glacomadataset2/glaucoma/image109prime.jpg  
  inflating: glacomadataset2/glaucoma/image10prime (2).jpg  
  inflating: glacomadataset2/glaucoma/image10prime.jpg  
  inflating: glacomadataset2/glaucoma/image110prime.jpg  
  inflating: glacomadataset2/glaucoma/image111prime.jpg  
  inflating: glacomadataset2/glaucoma/image112prime.jpg  
  infl

In [ ]:
import numpy as np
import pandas as pd
 
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
import pandocfilters as f
 
import os
import cv2
from google.colab import files
from glob import glob
from PIL import Image
from termcolor import colored
from sklearn.metrics import f1_score

In [ ]:
print(os.listdir("/content/glacomadataset2"))
train_transforms = transforms.Compose([transforms.Resize(size=(224,224)),
                                       transforms.ToTensor()])
validation_transforms = transforms.Compose([transforms.Resize(size=(224,224)),
                                            transforms.ToTensor()
                                            ])
img_dir = '/content/glacomadataset2'
train_data = datasets.ImageFolder(img_dir, transform=train_transforms)
# number of subprocesses to use for data loading
num_workers = 0
# percentage of training set to use as validation
valid_size = 0.2
test_size = 0.1
 
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor()
])
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
valid_split = int(np.floor((valid_size) * num_train))
test_split = int(np.floor((valid_size + test_size) * num_train))
valid_idx, test_idx, train_idx = indices[:valid_split], indices[valid_split:test_split], indices[test_split:]
 
print(len(valid_idx), len(test_idx), len(train_idx))
 
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)
 
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20,
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=20,
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=20,
                                          sampler=test_sampler, num_workers=num_workers)

['not_glaucoma', 'glaucoma']
194 97 680


In [ ]:
torch.save(train_loader, '/content/drive/MyDrive/glacoma1trainloader2.pth')
torch.save(valid_loader, '/content/drive/MyDrive/glacoma1validloader2.pth')
torch.save(test_loader, '/content/drive/MyDrive/glacoma1testloader2.pth')


In [ ]:
train_loader=torch.load('/content/drive/MyDrive/glacoma1trainloader2.pth')
valid_loader=torch.load('/content/drive/MyDrive/glacoma1validloader2.pth')
test_loader=torch.load('/content/drive/MyDrive/glacoma1testloader2.pth')

In [ ]:
model = models.resnet18(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False
 
 
num_ftrs=model.fc.in_features
 
#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))
 
model.fc= nn.Linear(num_ftrs, 2)
 
fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True
 
model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001,momentum=0.9)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def train_accuracy(model, criterion, use_cuda):
    # monitor test loss and accuracy
    #train_loss = 0.
    correct = 0.
    total = 0.
 
    for batch_idx, (data, target) in enumerate(train_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss
        #train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        train_accuracy=100. * correct / total
    print('\ntrain Accuracy: %2.2f%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    return train_accuracy

In [ ]:
 
def valid_accuracy(model, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
   
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss
        pred = output.data.max(1, keepdim=True)[1]
       
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        test_accuracy=100. * correct / total
    print('\n validation Accuracy: %2.2f%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
   
   
    return test_accuracy

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
 
 
def test(model, criterion, use_cuda):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    pre=0.0
    count=0
    rec=0.0
    f1_m=0.0
    au=0.0
    y_pred=[]
    y_target=[]
 
    for batch_idx, (data, target) in enumerate(test_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        y_pred_list = torch.tensor([a.squeeze().tolist() for a in pred])
        targetlist =torch.tensor([t.squeeze().tolist() for t in target])
       
        y_pred.append(y_pred_list)
        y_target.append(targetlist)
        #y_target = torch.stack(target,1) 
        
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        test_accuracy=100. * correct / total
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2.5f%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    y_pred_list = torch.cat(y_pred)
    targetlist = torch.cat(y_target)
    y_predd=y_pred_list.numpy()
    targett=targetlist.numpy()
    aucc=recall_score(targett, y_predd, average='macro')
   
    
    rec=recall_score(targett, y_predd, average='macro')
    pre =precision_score(targett, y_predd, average='macro')
    f_m=f1_score(targett, y_predd, average='macro')
 
    #auu=float(au/count)
    print("Precision="+str(pre))
    print("Recall="+str(rec))
    print("F1="+str(f_m))
    return test_accuracy
 
# %%

In [ ]:
def train(n_epochs, model, optimizer, criterion, use_cuda, save_path):
    
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs + 1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
 
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
 
            # initialize weights to zero
            optimizer.zero_grad()
            output = model(data)
            
            # calculate loss
            loss = criterion(output, target)
 
            # back prop
            loss.backward()
                      # grad
            optimizer.step()
 
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
 
            if batch_idx % 100 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                      (epoch, batch_idx + 1, train_loss))
 
        ######################
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(valid_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
 
        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
        ))
        
       
       
  
        
        
 
        #test(model, criterion, use_cuda)
        
        tra_ac = train_accuracy(model, criterion, use_cuda)
        val_acc = valid_accuracy(model, criterion, use_cuda)
     
        

        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            valid_loss_min = valid_loss
    
   

 
    return model
 

In [ ]:
train(25, model, optimizer, criterion, use_cuda, 'resnet18.pt')
model.load_state_dict(torch.load('resnet18.pt'))
test(model, criterion, use_cuda)

Epoch 1, Batch 1 loss: 0.735106
Epoch: 1 	Training Loss: 0.497804 	Validation Loss: 0.346919

train Accuracy: 89.85% (611/680)

 validation Accuracy: 87.11% (169/194)
Validation loss decreased (inf --> 0.346919).  Saving model ...
Epoch 2, Batch 1 loss: 0.197509
Epoch: 2 	Training Loss: 0.267360 	Validation Loss: 0.225269

train Accuracy: 94.85% (645/680)

 validation Accuracy: 93.30% (181/194)
Validation loss decreased (0.346919 --> 0.225269).  Saving model ...
Epoch 3, Batch 1 loss: 0.260514
Epoch: 3 	Training Loss: 0.228230 	Validation Loss: 0.188875

train Accuracy: 95.74% (651/680)

 validation Accuracy: 94.33% (183/194)
Validation loss decreased (0.225269 --> 0.188875).  Saving model ...
Epoch 4, Batch 1 loss: 0.129081
Epoch: 4 	Training Loss: 0.162980 	Validation Loss: 0.159343

train Accuracy: 97.50% (663/680)

 validation Accuracy: 95.36% (185/194)
Validation loss decreased (0.188875 --> 0.159343).  Saving model ...
Epoch 5, Batch 1 loss: 0.075706
Epoch: 5 	Training Loss: 0.15

96.90721649484536

In [ ]:
model = models.resnet50(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False


num_ftrs=model.fc.in_features

#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))

model.fc= nn.Linear(num_ftrs, 2)

fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True

model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'resnet50.pt')
model.load_state_dict(torch.load('resnet50.pt'))
test(model, criterion, use_cuda)

Epoch 1, Batch 1 loss: 0.704523
Epoch: 1 	Training Loss: 0.507096 	Validation Loss: 0.401308

train Accuracy: 89.26% (607/680)

 validation Accuracy: 89.18% (173/194)
Validation loss decreased (inf --> 0.401308).  Saving model ...
Epoch 2, Batch 1 loss: 0.219177
Epoch: 2 	Training Loss: 0.272071 	Validation Loss: 0.224146

train Accuracy: 94.71% (644/680)

 validation Accuracy: 92.27% (179/194)
Validation loss decreased (0.401308 --> 0.224146).  Saving model ...
Epoch 3, Batch 1 loss: 0.476140
Epoch: 3 	Training Loss: 0.214953 	Validation Loss: 0.193667

train Accuracy: 95.59% (650/680)

 validation Accuracy: 93.81% (182/194)
Validation loss decreased (0.224146 --> 0.193667).  Saving model ...
Epoch 4, Batch 1 loss: 0.133328
Epoch: 4 	Training Loss: 0.177163 	Validation Loss: 0.210722

train Accuracy: 95.74% (651/680)

 validation Accuracy: 92.78% (180/194)
Epoch 5, Batch 1 loss: 0.407887
Epoch: 5 	Training Loss: 0.177511 	Validation Loss: 0.159367

train Accuracy: 96.62% (657/680)

 v

98.96907216494846

In [ ]:
model = models.resnet101(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False


num_ftrs=model.fc.in_features

#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))

model.fc= nn.Linear(num_ftrs, 2)

fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True

model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'resnet101.pt')


Epoch 1, Batch 1 loss: 0.670438
Epoch: 1 	Training Loss: 0.505257 	Validation Loss: 0.355589

train Accuracy: 89.56% (609/680)

 validation Accuracy: 88.14% (171/194)
Validation loss decreased (inf --> 0.355589).  Saving model ...
Epoch 2, Batch 1 loss: 0.277255
Epoch: 2 	Training Loss: 0.241098 	Validation Loss: 0.219495

train Accuracy: 94.56% (643/680)

 validation Accuracy: 93.81% (182/194)
Validation loss decreased (0.355589 --> 0.219495).  Saving model ...
Epoch 3, Batch 1 loss: 0.237943
Epoch: 3 	Training Loss: 0.185726 	Validation Loss: 0.184971

train Accuracy: 97.06% (660/680)

 validation Accuracy: 92.27% (179/194)
Validation loss decreased (0.219495 --> 0.184971).  Saving model ...
Epoch 4, Batch 1 loss: 0.158490
Epoch: 4 	Training Loss: 0.159799 	Validation Loss: 0.177405

train Accuracy: 96.76% (658/680)

 validation Accuracy: 93.81% (182/194)
Validation loss decreased (0.184971 --> 0.177405).  Saving model ...
Epoch 5, Batch 1 loss: 0.161717
Epoch: 5 	Training Loss: 0.14

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model.load_state_dict(torch.load('resnet101.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.049116


Test Accuracy: 98.96907% (96/97)
Precision=0.9915254237288136
Recall=0.9871794871794872
F1=0.9892329892329892


98.96907216494846

In [ ]:
model = models.resnet152(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False
 
 
num_ftrs=model.fc.in_features
 
#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))
 
model.fc= nn.Linear(num_ftrs, 2)
 
fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True
 
model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'resnet152.pt')

Epoch 1, Batch 1 loss: 0.717934
Epoch: 1 	Training Loss: 0.484015 	Validation Loss: 0.411393

train Accuracy: 90.59% (616/680)

 validation Accuracy: 88.14% (171/194)
Validation loss decreased (inf --> 0.411393).  Saving model ...
Epoch 2, Batch 1 loss: 0.881498
Epoch: 2 	Training Loss: 0.283713 	Validation Loss: 0.269644

train Accuracy: 93.24% (634/680)

 validation Accuracy: 89.69% (174/194)
Validation loss decreased (0.411393 --> 0.269644).  Saving model ...
Epoch 3, Batch 1 loss: 0.259476
Epoch: 3 	Training Loss: 0.239944 	Validation Loss: 0.204056

train Accuracy: 93.53% (636/680)

 validation Accuracy: 94.33% (183/194)
Validation loss decreased (0.269644 --> 0.204056).  Saving model ...
Epoch 4, Batch 1 loss: 0.118185
Epoch: 4 	Training Loss: 0.158985 	Validation Loss: 0.150294

train Accuracy: 97.06% (660/680)

 validation Accuracy: 93.81% (182/194)
Validation loss decreased (0.204056 --> 0.150294).  Saving model ...
Epoch 5, Batch 1 loss: 0.098647
Epoch: 5 	Training Loss: 0.15

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model.load_state_dict(torch.load('resnet152.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.055729


Test Accuracy: 98.96907% (96/97)
Precision=0.9915254237288136
Recall=0.9871794871794872
F1=0.9892329892329892


98.96907216494846

In [ ]:
model = models.alexnet(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False


num_ftrs=model.classifier[6].in_features

#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))

model.classifier[6]= nn.Linear(num_ftrs, 2)

fc_parameters = model.classifier[6].parameters()
for param in fc_parameters:
    param.requires_grad = True

model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'alexnet.pt')


Epoch 1, Batch 1 loss: 1.279488
Epoch: 1 	Training Loss: 0.474686 	Validation Loss: 0.231276

train Accuracy: 93.53% (636/680)

 validation Accuracy: 91.24% (177/194)
Validation loss decreased (inf --> 0.231276).  Saving model ...
Epoch 2, Batch 1 loss: 0.133687
Epoch: 2 	Training Loss: 0.350779 	Validation Loss: 0.165906

train Accuracy: 95.59% (650/680)

 validation Accuracy: 93.30% (181/194)
Validation loss decreased (0.231276 --> 0.165906).  Saving model ...
Epoch 3, Batch 1 loss: 0.151756
Epoch: 3 	Training Loss: 0.194952 	Validation Loss: 0.154389

train Accuracy: 96.47% (656/680)

 validation Accuracy: 93.81% (182/194)
Validation loss decreased (0.165906 --> 0.154389).  Saving model ...
Epoch 4, Batch 1 loss: 0.030456
Epoch: 4 	Training Loss: 0.174247 	Validation Loss: 0.114885

train Accuracy: 96.91% (659/680)

 validation Accuracy: 95.36% (185/194)
Validation loss decreased (0.154389 --> 0.114885).  Saving model ...
Epoch 5, Batch 1 loss: 0.182820
Epoch: 5 	Training Loss: 0.14

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
model.load_state_dict(torch.load('alexnet.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.032692


Test Accuracy: 98.96907% (96/97)
Precision=0.9915254237288136
Recall=0.9871794871794872
F1=0.9892329892329892


98.96907216494846

In [ ]:
model = models.googlenet(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False


num_ftrs=model.fc.in_features

#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))

model.fc= nn.Linear(num_ftrs, 2)

fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.requires_grad = True

model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'googlenet.pt')


Epoch 1, Batch 1 loss: 0.732771
Epoch: 1 	Training Loss: 0.591894 	Validation Loss: 0.477058

train Accuracy: 81.47% (554/680)

 validation Accuracy: 79.90% (155/194)
Validation loss decreased (inf --> 0.477058).  Saving model ...
Epoch 2, Batch 1 loss: 0.451280
Epoch: 2 	Training Loss: 0.360547 	Validation Loss: 0.329654

train Accuracy: 90.15% (613/680)

 validation Accuracy: 87.63% (170/194)
Validation loss decreased (0.477058 --> 0.329654).  Saving model ...
Epoch 3, Batch 1 loss: 0.336085
Epoch: 3 	Training Loss: 0.295348 	Validation Loss: 0.288802

train Accuracy: 91.32% (621/680)

 validation Accuracy: 88.14% (171/194)
Validation loss decreased (0.329654 --> 0.288802).  Saving model ...
Epoch 4, Batch 1 loss: 0.336581
Epoch: 4 	Training Loss: 0.284162 	Validation Loss: 0.264443

train Accuracy: 91.47% (622/680)

 validation Accuracy: 90.21% (175/194)
Validation loss decreased (0.288802 --> 0.264443).  Saving model ...
Epoch 5, Batch 1 loss: 0.160299
Epoch: 5 	Training Loss: 0.24

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
model.load_state_dict(torch.load('googlenet.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.158632


Test Accuracy: 94.84536% (92/97)
Precision=0.9482604817127565
Recall=0.9442970822281167
F1=0.9461649461649462


94.84536082474227

In [ ]:
model = models.vgg11(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False
 
 
num_ftrs=model.classifier[6].in_features
 
#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))
 
model.classifier[6]= nn.Linear(num_ftrs, 2)
 
fc_parameters = model.classifier[6].parameters()
for param in fc_parameters:
    param.requires_grad = True
 
model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'vgg11.pt')


Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.cache/torch/hub/checkpoints/vgg11-bbd30ac9.pth



Epoch 1, Batch 1 loss: 0.661966
Epoch: 1 	Training Loss: 0.446897 	Validation Loss: 0.290380

train Accuracy: 87.94% (598/680)

 validation Accuracy: 87.11% (169/194)
Validation loss decreased (inf --> 0.290380).  Saving model ...
Epoch 2, Batch 1 loss: 0.322942
Epoch: 2 	Training Loss: 0.322384 	Validation Loss: 0.203490

train Accuracy: 91.47% (622/680)

 validation Accuracy: 91.75% (178/194)
Validation loss decreased (0.290380 --> 0.203490).  Saving model ...
Epoch 3, Batch 1 loss: 0.303631
Epoch: 3 	Training Loss: 0.289365 	Validation Loss: 0.182292

train Accuracy: 93.09% (633/680)

 validation Accuracy: 93.30% (181/194)
Validation loss decreased (0.203490 --> 0.182292).  Saving model ...
Epoch 4, Batch 1 loss: 0.280460
Epoch: 4 	Training Loss: 0.250027 	Validation Loss: 0.164016

train Accuracy: 95.15% (647/680)

 validation Accuracy: 94.33% (183/194)
Validation loss decreased (0.182292 --> 0.164016).  Saving model ...
Epoch 5, Batch 1 loss: 0.196898
Epoch: 5 	Training Loss: 0.2

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
model.load_state_dict(torch.load('vgg11.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.087695


Test Accuracy: 96.90722% (94/97)
Precision=0.969892952720785
Recall=0.9657382847038019
F1=0.9676989676989678


96.90721649484536

In [ ]:
model = models.vgg16(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False
 
 
num_ftrs=model.classifier[6].in_features
 
#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))
 
model.classifier[6]= nn.Linear(num_ftrs, 2)
 
fc_parameters = model.classifier[6].parameters()
for param in fc_parameters:
    param.requires_grad = True
 
model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'vgg16.pt')


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



Epoch 1, Batch 1 loss: 0.685569
Epoch: 1 	Training Loss: 0.465006 	Validation Loss: 0.275224

train Accuracy: 90.44% (615/680)

 validation Accuracy: 90.21% (175/194)
Validation loss decreased (inf --> 0.275224).  Saving model ...
Epoch 2, Batch 1 loss: 0.329962
Epoch: 2 	Training Loss: 0.292827 	Validation Loss: 0.242486

train Accuracy: 91.62% (623/680)

 validation Accuracy: 91.75% (178/194)
Validation loss decreased (0.275224 --> 0.242486).  Saving model ...
Epoch 3, Batch 1 loss: 0.367048
Epoch: 3 	Training Loss: 0.254685 	Validation Loss: 0.155563

train Accuracy: 94.41% (642/680)

 validation Accuracy: 95.88% (186/194)
Validation loss decreased (0.242486 --> 0.155563).  Saving model ...
Epoch 4, Batch 1 loss: 0.266138
Epoch: 4 	Training Loss: 0.222212 	Validation Loss: 0.141697

train Accuracy: 94.85% (645/680)

 validation Accuracy: 95.88% (186/194)
Validation loss decreased (0.155563 --> 0.141697).  Saving model ...
Epoch 5, Batch 1 loss: 0.232077
Epoch: 5 	Training Loss: 0.2

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
model.load_state_dict(torch.load('vgg16.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.072318


Test Accuracy: 98.96907% (96/97)
Precision=0.9875
Recall=0.9913793103448276
F1=0.9893230599889928


98.96907216494846

In [ ]:
model = models.vgg19(pretrained=True)
#12 30
for param in model.parameters():
    param.requires_grad = False
 
 
num_ftrs=model.classifier[6].in_features
 
#model.classifier[1]=nn.Conv2d(512,2,kernel_size=(1,1), stride=(1,1))
 
model.classifier[6]= nn.Linear(num_ftrs, 2)
 
fc_parameters = model.classifier[6].parameters()
for param in fc_parameters:
    param.requires_grad = True
 
model
use_cuda = torch.cuda.is_available()
if use_cuda:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001,momentum=0.9)
train(25, model, optimizer, criterion, use_cuda, 'vgg19.pt')


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



Epoch 1, Batch 1 loss: 0.673606
Epoch: 1 	Training Loss: 0.431861 	Validation Loss: 0.248724

train Accuracy: 88.09% (599/680)

 validation Accuracy: 89.18% (173/194)
Validation loss decreased (inf --> 0.248724).  Saving model ...
Epoch 2, Batch 1 loss: 0.178911
Epoch: 2 	Training Loss: 0.259631 	Validation Loss: 0.177445

train Accuracy: 93.38% (635/680)

 validation Accuracy: 92.27% (179/194)
Validation loss decreased (0.248724 --> 0.177445).  Saving model ...
Epoch 3, Batch 1 loss: 0.264376
Epoch: 3 	Training Loss: 0.227441 	Validation Loss: 0.152263

train Accuracy: 94.41% (642/680)

 validation Accuracy: 95.88% (186/194)
Validation loss decreased (0.177445 --> 0.152263).  Saving model ...
Epoch 4, Batch 1 loss: 0.594169
Epoch: 4 	Training Loss: 0.237262 	Validation Loss: 0.147996

train Accuracy: 93.97% (639/680)

 validation Accuracy: 95.88% (186/194)
Validation loss decreased (0.152263 --> 0.147996).  Saving model ...
Epoch 5, Batch 1 loss: 0.224884
Epoch: 5 	Training Loss: 0.1

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
model.load_state_dict(torch.load('vgg19.pt'))
test(model, criterion, use_cuda)

Test Loss: 0.102882


Test Accuracy: 95.87629% (93/97)
Precision=0.95448606271777
Recall=0.9613174182139699
F1=0.9574561403508772


95.87628865979381